In [128]:
from autotrader import AutoData
import json
import requests
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

#------------------------------------------------------------------------------------
'''Pulling Mycelium funding rate data to csv.'''
#pull historical funding rate and price data
myc_fr_response = requests.get("https://dev.api.tracer.finance/trs/fundingRates")
myc_fr_data = pd.DataFrame(myc_fr_response.json()['rows'])

#write funding rate data to csv
myc_fr_data.to_csv('raw_data/myc_fr.csv')

#------------------------------------------------------------------------------------
'''Calculate times for pulling rest of the mycelium price, binance perp funding rate and price data.'''

#get first and last funding rate data times, used for mycelium price data
start_unix_str = myc_fr_data['timestamp'].iloc[-1]
end_unix_str = myc_fr_data['timestamp'].iloc[0]

start_datetime = pd.to_datetime(end_unix_str, unit = 's').to_pydatetime()
end_datetime = pd.to_datetime(end_unix_str, unit = 's').to_pydatetime()

#------------------------------------------------------------------------------------
'''Pulling mycelium price data to csv.'''

#page size for price data
page_size = 1000

#get the number of data entries and first page of data
price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page=1&pageSize{page_size}')
price_entries = price_response.json()['totalRecords']
myc_price = pd.DataFrame(price_response.json()['rows'])

#function used in threads to pull price data
def pull_myc_price(start_unix_str, end_unix_str, page_size, page_no):
    price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page={page_no}&pageSize{page_size}')
    price_data = pd.DataFrame(price_response.json()['rows'])
    return(price_data)

#create threads
with ThreadPoolExecutor(max_workers=6) as executor:
    price_list=[]
    for i in range(2, (price_entries//page_size)+2):
        # submit job:
        price_list.append(executor.submit(
            pull_myc_price,
            start_unix_str = start_unix_str,
            end_unix_str = end_unix_str,
            page_size = page_size,
            page_no = i
        ))

#interpret results of threading   
for i,val in enumerate(price_list):
    result = price_list[i].result()
    myc_price = pd.concat([myc_price,result])

#write price data to csv
myc_price.to_csv('raw_data/myc_price.csv')

#------------------------------------------------------------------------------------
'''Pulling binance perp funding rate and price data to csv.'''
#the following are the myc universe assets that have perps on binance, ETH = WETH, BTC = WBTC
binance_universe = ['ETH','BTC','LINK','UNI','FXS','BAL','CRV']

# perps on binance come with USDT or BUSD, so both will be used to compare
perp_quote = ['USDT', 'BUSD']

binance = AutoData(data_source ='ccxt', exchange = 'binanceusdm')
for quote in perp_quote:
    for ticker in binance_universe:
        #pull data into data frames
        try:
            fr_data = binance._ccxt_funding_history(f'{ticker}{quote}', start_time=start_datetime, end_time=end_datetime)
        except:
            continue
        try:
            price_data = binance.fetch(f'{ticker}{quote}', granularity='8h', start_time=start_datetime, end_time=end_datetime)
        except:
            continue
        
        #store data in csv's
        fr_data.to_csv(f'raw_data/binance_fr/{ticker}{quote}.csv')
        price_data.to_csv(f'raw_data/binance_price/{ticker}{quote}.csv')

#------------------------------------------------------------------------------------

#remove token addresses
myc_fr_data = myc_fr_data.drop(['token'], axis =1)



#define funding rate columns
fr = (myc_fr_data.endFundingRate.astype(int) - myc_fr_data.startFundingRate.astype(int))/1000000

time = pd.to_datetime(myc_fr_data['timestamp'], unit = 's')
token = myc_fr_data.symbol

myc_funding = pd.concat([time,token,fr], axis=1)
myc_funding = myc_funding.rename(columns={0:'funding_rate'})





print(myc_funding)

               timestamp symbol  funding_rate
0    2022-11-11 02:00:00   USDT      0.000000
1    2022-11-11 02:00:00    CRV      0.000000
2    2022-11-11 02:00:00   LINK      0.000007
3    2022-11-11 02:00:00    DAI      0.000000
4    2022-11-11 02:00:00   WETH      0.000001
...                  ...    ...           ...
7000 2022-10-01 14:00:00    DAI      0.000000
7001 2022-10-01 14:00:00   USDT      0.000000
7002 2022-10-01 13:00:00   LINK      0.000023
7003 2022-10-01 13:00:00   USDT      0.000000
7004 2022-10-01 13:00:00   USDC      0.000001

[7005 rows x 3 columns]


In [147]:
import pandas as pd

#------------------------------------------------------------------------------------
''''Processing Mycelium funding rate data'''
#mycelium universe
myc_universe = ['WETH','WBTC','LINK','UNI','USDC','USDT','DAI','FRAX','FXS','BAL','CRV']

#read raw data from csv
myc_fr_raw = pd.read_csv('raw_data/myc_fr.csv', 
        usecols=['timestamp','symbol','endFundingRate','startFundingRate']
) 

#define funding rate columns
symbol = myc_fr_raw.symbol
timestamp = pd.to_datetime(myc_fr_raw['timestamp'], unit = 's')
fr = (myc_fr_raw.endFundingRate.astype(int) - myc_fr_data.startFundingRate.astype(int))/1000000

#create data frame
myc_fr = pd.concat([timestamp,symbol,fr], axis=1)
myc_fr = myc_fr.rename(columns={0:'funding_rate'}) #label funding rate column
myc_fr = myc_fr.iloc[::-1]

token_data = []
for ticker in myc_universe:
        data = myc_fr[myc_fr.symbol == ticker]
        data_dict = {
                'token': f'{ticker}',
                'data': data,
        }
        print(data)
        token_data.append(data_dict)

        




               timestamp symbol  funding_rate
6999 2022-10-01 14:00:00   WETH      0.000004
6992 2022-10-01 15:00:00   WETH      0.000004
6983 2022-10-01 16:00:00   WETH      0.000004
6973 2022-10-01 17:00:00   WETH      0.000004
6965 2022-10-01 18:00:00   WETH      0.000004
...                  ...    ...           ...
39   2022-11-10 22:00:00   WETH      0.000000
26   2022-11-10 23:00:00   WETH      0.000000
19   2022-11-11 00:00:00   WETH      0.000000
14   2022-11-11 01:00:00   WETH      0.000000
4    2022-11-11 02:00:00   WETH      0.000001

[954 rows x 3 columns]
               timestamp symbol  funding_rate
6990 2022-10-01 15:00:00   WBTC      0.000011
6984 2022-10-01 16:00:00   WBTC      0.000003
6972 2022-10-01 17:00:00   WBTC      0.000003
6962 2022-10-01 18:00:00   WBTC      0.000003
6943 2022-10-01 21:00:00   WBTC      0.000011
...                  ...    ...           ...
37   2022-11-10 22:00:00   WBTC      0.000000
27   2022-11-10 23:00:00   WBTC      0.000000
18   2022-

In [132]:
start_unix_str = myc_fr_data['timestamp'].iloc[-1]
end_unix_str = myc_fr_data['timestamp'].iloc[0]

start_unix = pd.to_datetime(end_unix_str, unit = 's').to_pydatetime()
end_unix = pd.to_datetime(end_unix_str, unit = 's').to_pydatetime()

print(start_unix)
print(type(start_unix))

2022-11-11 02:00:00
<class 'datetime.datetime'>


In [109]:
import numpy as np
page_size = 100
price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page=1&pageSize{page_size}')
price_entries = price_response.json()['totalRecords']
myc_price = pd.DataFrame(price_response.json()['rows'])

for i in range(2,(price_entries//100)+2):
    price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page={i}&pageSize{page_size}')
    price_data = pd.DataFrame(price_response.json()['rows'])
    myc_price = pd.concat([myc_price, price_data])

#remove headers that aren't required
myc_price = myc_price.drop(['token','txnHash','blockNumber'], axis =1)

#refactor prices to USD
myc_price['price'] = myc_price['price'].str.removesuffix('000000000000000000000000000') # removes 27 zero's
myc_price['price'] = myc_price['price'].astype({'price':'int'})/1000


#Change UNIX time string objects to datetime objects
myc_price['timestamp'] = pd.to_datetime(myc_price['timestamp'], unit = 's')
myc_price.to_csv('myc_price_data/price_data.csv')

print(len(myc_price.price))
print(myc_price.to_markdown())



KeyboardInterrupt: 

In [118]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import json
import requests

page_size = 1000

price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page=1&pageSize{page_size}')
price_entries = price_response.json()['totalRecords']
myc_price = pd.DataFrame(price_response.json()['rows'])

def pull_myc_price(start_unix_str, end_unix_str, page_size, page_no):
    price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page={page_no}&pageSize{page_size}')
    price_data = pd.DataFrame(price_response.json()['rows'])
    return(price_data)

with ThreadPoolExecutor(max_workers=6) as executor:
    price_list=[]
    for i in range(2, (price_entries//page_size)+2):
        # submit job:
        price_list.append(executor.submit(
            pull_myc_price,
            start_unix_str = start_unix_str,
            end_unix_str = end_unix_str,
            page_size = page_size,
            page_no = i
        ))
    
for i,val in enumerate(price_list):
    result = price_list[i].result()
    myc_price = pd.concat([myc_price,result])

myc_price.to_csv('raw_data/myc_price.csv')


In [127]:
#the following are the myc universe assets that have perps on binance, ETH = WETH, BTC = WBTC
binance_universe = ['ETH','BTC','LINK','UNI','FXS','BAL','CRV']

# perps on binance come with USDT or BUSD, so both will be used to compare
perp_quote = ['USDT', 'BUSD']

#start time determined by the earliest recorded funding rate from Mycelium
start = start_unix.to_pydatetime()
end = end_unix.to_pydatetime()

binance = AutoData(data_source ='ccxt', exchange = 'binanceusdm')
for quote in perp_quote:
    for ticker in binance_universe:
        #pull data into data frames
        try:
            fr_data = binance._ccxt_funding_history(f'{ticker}{quote}', start_time=start, end_time=end)
        except:
            continue
        try:
            price_data = binance.fetch(f'{ticker}{quote}', granularity='8h', start_time=start, end_time=end)
        except:
            continue
        
        #store data in csv's
        fr_data.to_csv(f'raw_data/binance_fr/{ticker}{quote}.csv')
        price_data.to_csv(f'raw_data/binance_price/{ticker}{quote}.csv')

In [ ]:
fr_data = response.json()['rows']
df_data = pd.DataFrame(fr_data)
df_data = df_data.drop(['token','endFundingRate'], axis = 1)
print(df_data.to_markdown())

#for i in fr_data['rows']:
#    print(i)


|      | symbol   |   startFundingRate |        utilization |   timestamp |
|-----:|:---------|-------------------:|-------------------:|------------:|
|    0 | UNI      |              15394 |                  0 |  1667973600 |
|    1 | LINK     |              39855 |                  0 |  1667973600 |
|    2 | FXS      |              35011 |                  0 |  1667973600 |
|    3 | WETH     |              28677 |  35694108220480980 |  1667973600 |
|    4 | USDC     |              15811 |                  0 |  1667973600 |
|    5 | USDT     |               4604 |                  0 |  1667973600 |
|    6 | USDT     |               4604 |                  0 |  1667970000 |
|    7 | USDC     |              15801 | 207523239815432626 |  1667970000 |
|    8 | FRAX     |               1082 |                  0 |  1667970000 |
|    9 | CRV      |               3935 |  15430875678252447 |  1667970000 |
|   10 | FXS      |              35011 |                  0 |  1667970000 |
|   11 | WET